<a href="https://colab.research.google.com/github/ct-williams/TCBLNet/blob/main/CS230_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
import h5py

In [2]:
# Load Data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Deep Learning Approach

  # Standard Layer Definition
class StandardLayer(nn.Module):
  def __init__(self, n_input_features, n_output_features):
    super(StandardLayer, self).__init__()
    self.lin1 = nn.Linear(n_input_features, n_input_features)
    self.lin2 = nn.Linear(n_input_features, n_output_features)
    self.LRelu = nn.LeakyReLU(0.01)

  def forward(self, x):
    z1 = self.LRelu(self.lin1((x)))
    z2 = self.LRelu(self.lin2(z1)+self.lin2(x))
    return z2

# Model Definition
Model = nn.Sequential(   # Layer 0
                      StandardLayer(55,40),  # Layer 1
                      StandardLayer(40,40),  # Layer 5
                      StandardLayer(40,20),  # Layer 6
                      StandardLayer(20,20),  # Layer 10
                      StandardLayer(20,10),  # Layer 12
                      StandardLayer(10,10),  # Layer 15
                      nn.Linear(10, 2))      # Layer 11 - Output Layer
Model.double()

# Loss Function 
LossFunction = nn.MSELoss()

# Optimizer 
Optimizer = torch.optim.Adam(Model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)

In [ ]:
# Training
num_epochs = 25
num_batches = 3600
train_dir = "/content/drive/MyDrive/training_data/training_selection_2/"

for epoch in range(num_epochs):
  with torch.no_grad():
     norm_error = 0
  for batch in range(1, num_batches+1): 
     # Load data from batch (i) 
     filename = train_dir+"batch_"+ str(batch).zfill(5) + ".hdf"
     f = h5py.File(filename, 'r')
     data = np.transpose(f['data'])
     Y = data[:,0:2]
     X = data[:,2:57]
     # Convert numpy arrays into tensors
     inputs = torch.from_numpy(X)
     labels = torch.from_numpy(Y)
     # Forward propagation
     predictions = Model(inputs)
     cost = LossFunction(predictions, labels)
     # Backward propagation
     Optimizer.zero_grad()
     cost.backward()
     Optimizer.step()
     with torch.no_grad():
       norm_error += cost.item()
  if (epoch)%5 == 0:
     print(norm_error/(num_batches))

62.54079265012157
42.825202612716645
40.472860997945816
39.57347870033841
38.97571081749613


In [ ]:
# Error Evaluation of Model on Train Data
with torch.no_grad():
   tau_error = 0
   q_error = 0
   for j in range(1, num_batches+1):
      # Load test data from batch (i) into numpy here
      filename = train_dir+"batch_"+ str(j).zfill(5) + ".hdf"
      f = h5py.File(filename, 'r')
      data = np.transpose(f['data'])
      Y_tau_np = data[:,0]
      Y_q_np = data[:,1]
      X_np = data[:,2:57]
      X = torch.from_numpy(X_np)
      Y_tau = torch.from_numpy(Y_tau_np)
      Y_q = torch.from_numpy(Y_q_np)
      Y_hat = Model(X)
      Y_tau_hat = Y_hat[:,0]
      Y_q_hat = Y_hat[:,1]
      q_error += 2*torch.mean(torch.div(torch.abs(Y_q-Y_q_hat),torch.add(torch.abs(Y_q),torch.abs(Y_q_hat))))
      tau_error += 2*torch.mean(torch.div(torch.abs(Y_tau-Y_tau_hat),torch.add(torch.abs(Y_tau),torch.abs(Y_tau_hat))))
   tau_error /= num_batches
   q_error /= num_batches
print("Training q error: ", q_error.item())
print("Training tau error: ", tau_error.item())

Training q error:  0.23550147652738468
Training tau error:  0.6796437163914211


In [ ]:
# Error Evaluation of Model on Dev Data
num_dev_batches = 900
dev_dir = "/content/drive/MyDrive/training_data/test_selection_2/"
with torch.no_grad():
   norm_error = 0
   for j in range(1, num_dev_batches+1):
      # Load test data from batch (i) into numpy here
      filename = dev_dir+"batch_"+ str(j).zfill(5) + ".hdf"
      f = h5py.File(filename, 'r')
      data = np.transpose(f['data'])
      Y_tau_np = data[:,0]
      Y_q_np = data[:,1]
      X_np = data[:,2:57]
      X = torch.from_numpy(X_np)
      Y_tau = torch.from_numpy(Y_tau_np)
      Y_q = torch.from_numpy(Y_q_np)
      Y_hat = Model(X)
      Y_tau_hat = Y_hat[:,0]
      Y_q_hat = Y_hat[:,1]
      q_error += 2*torch.mean(torch.div(torch.abs(Y_q-Y_q_hat),torch.add(torch.abs(Y_q),torch.abs(Y_q_hat))))
      tau_error += 2*torch.mean(torch.div(torch.abs(Y_tau-Y_tau_hat),torch.add(torch.abs(Y_tau),torch.abs(Y_tau_hat))))
   tau_error /= num_dev_batches
   q_error /= num_dev_batches
print("Dev q error: ", q_error.item())
print("Dev tau error: ", tau_error.item())

Dev q error:  0.30782937874369015
Dev tau error:  0.7826910869954976


In [ ]:
# Error Evaluation of Model on Test Data
test_dir = "/content/drive/MyDrive/training_data/test_selection_2/"
with torch.no_grad():
   norm_error = 0
   for j in range(num_dev_batches+1, 2*num_dev_batches+1):
      # Load test data from batch (i) into numpy here
      filename = test_dir+"batch_"+ str(j).zfill(5) + ".hdf"
      f = h5py.File(filename, 'r')
      data = np.transpose(f['data'])
      Y_tau_np = data[:,0]
      Y_q_np = data[:,1]
      X_np = data[:,2:57]
      X = torch.from_numpy(X_np)
      Y_tau = torch.from_numpy(Y_tau_np)
      Y_q = torch.from_numpy(Y_q_np)
      Y_hat = Model(X)
      Y_tau_hat = Y_hat[:,0]
      Y_q_hat = Y_hat[:,1]
      q_error += 2*torch.mean(torch.div(torch.abs(Y_q-Y_q_hat),torch.add(torch.abs(Y_q),torch.abs(Y_q_hat))))
      tau_error += 2*torch.mean(torch.div(torch.abs(Y_tau-Y_tau_hat),torch.add(torch.abs(Y_tau),torch.abs(Y_tau_hat))))
   tau_error /= num_dev_batches
   q_error /= num_dev_batches
print("Test q error: ", q_error.item())
print("Test tau error: ", tau_error.item())

Test q error:  0.31000395016604754
Test tau error:  0.7830040029207743
